 <img src="Logo casa 2022 negro.png" width="300" height="300">


In [18]:
# jupyter nbconvert --to html --no-input Mapa_Puntos.ipynb convertir a html sin codigo

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import folium

In [2]:
import geopandas as gpd  #importar libreria geopandas
geodata=gpd.read_file("https://www.medellin.gov.co/mapas/rest/services/ServiciosPlaneacion/POT48_Base/MapServer/3/query?where=1%3D1&outFields=*&outSR=4326&f=json")#archivo geo Json con la geometria de las comunnas de Medellin

In [39]:
df=pd.read_excel('Segunda vuelta.xlsx')
df[['Lat','Lon']]=df.Locacion.str.split(',',expand=True)

In [40]:
def redondear(x):
    return round(x,2)


df['Pacto']=(df['Pacto']*100).apply(redondear)
df['Pacto'] = df['Pacto'].astype(str)+'%'
df['Gobernantes']=(df['Gobernantes']*100).apply(redondear)
df['Gobernantes'] = df['Gobernantes'].astype(str)+'%'


In [41]:
df['Nombre'] = df['Nombre'].replace({'COL':'Col. '}, regex=True)
#df['Nombre'].head(20)

In [42]:
df['Nombre']=df['Nombre'].str.title()
#df['Nombre']

In [43]:
filtro_Petro=df[df['Pacto Histórico']>=df['Liga de Gobernantes Anticorrupción']]
filtro_Rodolfo=df[df['Pacto Histórico']<df['Liga de Gobernantes Anticorrupción']]

In [44]:
m = folium.Map(location=[6.2491965599999997,-75.5619602900000018], zoom_start=12) #inicializa el mapa de Medellín

#Graficar comunas de Medellin
group0 = folium.FeatureGroup(name='')
for _, r in geodata.iterrows(): #mapa de los bordes de cada comuna de Medellin
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.0001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'grey','color': 'black',
                                                          'weight': 1})
    folium.Popup('<b>{}</b>'.format(r['NOMBRE'])).add_to(geo_j)#añade un recuadro con el nombre de cada comuna
    geo_j.add_to(group0)
    
    
group0.add_to(m) 
group1 = folium.FeatureGroup(name='Petro')
for _,r in filtro_Petro.iterrows():
    popContent = ('<b>Lugar:</b> {} <br> <b>Gustavo Petro:</b> {} <br> <b>Rodolfo Hernández:</b> {} '.format(r['Nombre'],r['Pacto'],r['Gobernantes']))
    iframe = folium.IFrame(popContent)
    popup1 = folium.Popup(iframe,
                         min_width=200,
                         max_width=300)   
    folium.Marker(location=[r['Lat'], r['Lon']],popup = popup1,icon= folium.Icon(color='purple')).add_to(group1)
group1.add_to(m)


group2 = folium.FeatureGroup(name='Rodolfo')
for _,r in filtro_Rodolfo.iterrows():
    popContent = ('<b>Lugar:</b> {} <br> <b>Gustavo Petro:</b> {} <br> <b>Rodolfo Hernández:</b> {} '.format(r['Nombre'],r['Pacto'],r['Gobernantes']))
    iframe = folium.IFrame(popContent)
    popup1 = folium.Popup(iframe,
                         min_width=200,
                         max_width=300)   
    folium.Marker(location=[r['Lat'], r['Lon']],popup = popup1,icon= folium.Icon(color='orange')).add_to(group2)
group2.add_to(m)

folium.map.LayerControl('topright').add_to(m)
m
